In [24]:
import csv
import ast
import numpy as np
from collections import Counter
from sklearn import preprocessing
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from keras import backend as K

In [25]:
def evaluate_model(shape,mlb,BS,model,NUM_TEST_IMAGES,testLabels,TEST_CSV,number):
    # re-initialize our testing data generator, this time for evaluating
    testGen = csv_image_generator_multilabel_predictor(shape,TEST_CSV, BS)
    #predict_generator(generator, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)
    print("RUNNING PREDICTIONS")
    predictions = model.predict_generator(testGen, steps=NUM_TEST_IMAGES/1,workers=1,use_multiprocessing=True ,verbose=1)
    write_Accuracy(predictions,testLabels,number,mlb)
    predictions = []
    Y_Test = []
    
def write_Accuracy(predictions,Y_Test,number,mlb):
    recalls = get_recalls_for_k(predictions,Y_Test,4,len(mlb.classes_))
    
    row = ["Classes","k = 1 ","k = 2","k = 3","k = 4"]
    rows = []
    rows.append(row)
    for i in range(len(mlb.classes_)):
        row  = [mlb.classes_[i],recalls[0][i],recalls[1][i],recalls[2][i],recalls[3][i]]
        rows.append(row)    
    
    rows.append(["Micro AVG",sum(recalls[0])/len(recalls[0]),sum(recalls[1])/len(recalls[1]),sum(recalls[2])/len(recalls[2]),sum(recalls[3])/len(recalls[3])])
    location = "GeometricReports/"+str(number)+".csv"
    with open(location,"a",newline="") as f1: 
        cw = csv.writer(f1)
        cw.writerows(rows)
    f1.close()
    recalls = []
    rows = []
    predictions = []
    Y_Test = []

In [26]:
def y_count_1(a):
    m = len(a)
    y_count = []
    for i in range(m):
        #print(a[i])
        if a[i] == 1:
            y_count.append(i)
    return y_count

def get_pred_rank1(a):
    N=len(a)
    b=np.zeros([N],dtype=[('prob',float),('idx',int)])
    #for i in range(N)
    b['idx']=np.arange(N)
    b['prob']=np.copy(a)
    b.sort(order='prob')
    return b

def TP_FN(y_pred,y_test,k):
    TP = []
    FN = []
    for j in range(len(y_pred)):
        Y = y_test[j]
        H = get_pred_rank1(y_pred[j])        
        y_count = y_count_1(y_test[j])
        y_rank = np.flip(np.asarray(H['idx'],dtype=int),0) 
        #print(y_rank)
        for p in y_count:
            if p in y_rank[:k]:
                TP.append(p)
            else:
                #print(p)
                FN.append(p)
            
    return TP,FN 

def get_per_label(TP,FN,n):
    Recalls = []
    for i in range(0,n):
        TP1 = TP.get(i)
        FN1 = FN.get(i)
        if TP1 == None:
            TP1 = 0
        if FN1 == None:
            FN1 = 0
                
        if (TP1+FN1) == 0:
            Recall = 0
        else:
            Recall  = (TP1)/(TP1+FN1)
            
        Recalls.append(Recall)
        
    return Recalls

#Recall => TP/TP+FN
def get_recalls_for_k(predictions,Y_Test,k,n):
    ALL_Recalls = []
    for i in range(k): 
        TP,FN  = TP_FN(predictions,Y_Test,i+1)
        TP_Counter = Counter(TP)
        FN_Counter = Counter(FN)
        Recall = get_per_label(TP_Counter,FN_Counter,n)
        ALL_Recalls.append(Recall)
    return ALL_Recalls

In [16]:
def prediction_labels(a,p,k):
    #a is the list of percentages in order 
    #p is the percentage cut off point e.g 50%
    #k is the top k answers to take
    pred_l = []
    temp_index = []
    count = 0
    m = len(a)
    a = a.tolist()
    for i in a:
        if count <= k and (max(a)*100) >= p:
            highest = max(a)
            temp_index.append(a.index(highest))
            a.remove(highest)
            count += 1 
    for i in range(m):
        pred_l.append(0)
    for j in range(len(temp_index)):
        pred_l[temp_index[j]] = 1
                
    return pred_l

def prediction_labels_FCFS(a,p,k):
    #a is the list of percentages in order 
    #p is the percentage cut off point e.g 50%
    #k is the top k answers to take
    pred_l = []
    temp_top = []
    count = 0
    #Problem this is FCFS
    for i in range(len(a)):
        if (a[i]*100) >= p and count <= k:
            count += 1
            pred_l.append(1)
        elif (a[i]*100) <= p:
            pred_l.append(0)
        else:
            pred_l.append(0)
    return pred_l


def ordered_intersection(y, h):
    lst3 = []
    y_count = 0
    h_count = 0
    for i in range(len(y)):
        if y[i] == 1: 
            y_count +=1 
        if h[i] == 1:
            h_count +=1 
        if y[i] == h[i] and y[i] == 1:
            lst3.append(y[i])
    return lst3 , y_count,h_count

#https://stackoverflow.com/questions/9004172/precision-recall-for-multiclass-multilabel-classification
#Example based Metrics for Precision and Recall
def precision_recall(y_pred,y_test):
    pres = []
    rec  = []
    acc  = []
    n = len(y_pred)
    for i in range(len(y_pred)):
        lst3 , y_count,h_count = ordered_intersection(y_test[i], y_pred[i])
        if h_count == 0:
            pres.append(0)
            rec.append((len(lst3)/y_count))
        elif y_count == 0:
            pres.append((len(lst3)/h_count))
            rec.append(0)
        else:
            pres.append((len(lst3)/h_count))
            rec.append((len(lst3)/y_count))
        acc.append(len(lst3)/(y_count+h_count))
        
    precision = sum(pres)/n
    recall = sum(rec)/n
    accuracy = sum(acc)/n
    B = 1 
    F1 = ((1+B**2)*precision*recall)/((B**2)*precision+recall)
    
    return precision,recall,accuracy,F1

def TP_FP_TN_FN(y_pred,y_test):
    TP = []
    FP = []
    TN = []
    FN = []
    
    for j in range(len(y_pred)):
        H = y_pred[j]
        Y = y_test[j]

        for i in range(len(Y)):
            if Y[i] == H[i] and Y[i] == 1:
                TP.append(i)
            elif Y[i] != H[i]:
                if Y[i] == 1:
                    FN.append(i)
                elif H[i] == 1:
                    FP.append(i)
            elif Y[i] == H[i] and Y[i] == 0:
                TN.append(i)
                
    return TP,FP,TN,FN
    
def Macro_Labels(TP,FP,TN,FN,n):
    Accuracies = []
    Precisions = []
    Recalls    = []
    F1s        = []
    B = 1 
    for i in range(0,n):
        TP1 = TP.get(i)
        FP1 = FP.get(i)
        TN1 = TN.get(i)
        FN1 = FN.get(i)
        #print(TP1,FP1,TN1,FN1)
        for i1 in range(4):
            if TP1 == None:
                TP1 = 0
            elif FP1 == None:
                FP1 = 0
            elif TN1 == None:
                TN1 = 0
            elif FN1 == None:
                FN1 = 0
        
        if (TP1+FP1+TN1+FN1) == 0:
            Accuracy = 0
        else:
            Accuracy  = (TP1+TN1)/(TP1+FP1+TN1+FN1)
        
        if (TP1+FP1) == 0:
            Precision = 0
        else:
            Precision = (TP1)/(TP1+FP1)
        
        if (TP1+FN1) == 0:
            Recall = 0
        else:
            Recall    = (TP1)/(TP1+FN1)
        
        if ((1+B**2)*TP1+(B**2+FN1)+FP1) == 0:
            F1 = 0
        else:
            F1  = (TP1*(1+(B**2)))/((1+B**2)*TP1+(B**2+FN1)+FP1)
        
        Accuracies.append(Accuracy)
        Precisions.append(Precision)
        Recalls.append(Recall)
        F1s.append(F1)
        
        Ma_Acc  = sum(Accuracies)/n
        Ma_Pres = sum(Precisions)/n
        Ma_Rec  = sum(Recalls)/n
        Ma_F1     = sum(F1s)/n
    
    return Ma_Acc,Ma_Pres,Ma_Rec,Ma_F1

def Micro_Labels(TP,FP,TN,FN,n):
    B = 1
    All_TP = []
    All_FP = []
    All_TN = []
    All_FN = []

    for i in range(0,n):
        TP1 = TP.get(i)
        FP1 = FP.get(i)
        TN1 = TN.get(i)
        FN1 = FN.get(i)
        #print(TP1,FP1,TN1,FN1)
        for i1 in range(4):
            if TP1 == None:
                TP1 = 0
            elif FP1 == None:
                FP1 = 0
            elif TN1 == None:
                TN1 = 0
            elif FN1 == None:
                FN1 = 0

        All_TP.append(TP1)
        All_FP.append(FP1)
        All_TN.append(TN1)
        All_FN.append(FN1)

    TP1 = sum(All_TP)
    FP1 = sum(All_FP)
    TN1 = sum(All_TN)
    FN1 = sum(All_FN)

    if (TP1+FP1+TN1+FN1) == 0:
        Accuracy = 0
    else:
        Accuracy  = (TP1+TN1)/(TP1+FP1+TN1+FN1)

    if (TP1+FP1) == 0:
        Precision = 0
    else:
        Precision = (TP1)/(TP1+FP1)

    if (TP1+FN1) == 0:
        Recall = 0
    else:
        Recall    = (TP1)/(TP1+FN1)

    if ((1+B**2)*TP1+(B**2+FN1)+FP1) == 0:
        F1 = 0
    else:
        F1 = (TP1*(1+(B**2)))/((1+B**2)*TP1+(B**2+FN1)+FP1)

    Mi_Acc = Accuracy
    Mi_Pres = Precision
    Mi_Rec = Recall
    Mi_F1 = F1

    return Mi_Acc,Mi_Pres,Mi_Rec,Mi_F1

def get_per_label1(TP,FP,TN,FN,n):
    B = 1
    Accuracies = []
    Precisions = []
    Recalls    = []
    F1s        = []
    for i in range(0,n):
        TP1 = TP.get(i)
        FP1 = FP.get(i)
        TN1 = TN.get(i)
        FN1 = FN.get(i)
        #print(TP1,FP1,TN1,FN1)
        for i1 in range(4):
            if TP1 == None:
                TP1 = 0
            elif FP1 == None:
                FP1 = 0
            elif TN1 == None:
                TN1 = 0
            elif FN1 == None:
                FN1 = 0

        if (TP1+FP1+TN1+FN1) == 0:
            Accuracy = 0
        else:
            Accuracy  = (TP1+TN1)/(TP1+FP1+TN1+FN1)

        if (TP1+FP1) == 0:
            Precision = 0
        else:
            Precision = (TP1)/(TP1+FP1)

        if (TP1+FN1) == 0:
            Recall = 0
        else:
            Recall    = (TP1)/(TP1+FN1)

        if ((1+B**2)*TP1+(B**2+FN1)+FP1) == 0:
            F1 = 0
        else:
            F1 = (TP1*(1+(B**2)))/((1+B**2)*TP1+(B**2+FN1)+FP1)
        
        #print(Accuracy,Precision,Recall,F1)
        Accuracies.append(Accuracy)
        Precisions.append(Precision)
        Recalls.append(Recall)
        F1s.append(F1)
    
    return Accuracies,Precisions,Recalls,F1s
    
    
def get_recall_at_K_Example(y_pred,y_test,k,p,n):
    predictions = []
    predictions2 = []
    for i in range(len(y_test)):
        pred_l = prediction_labels(y_pred[i],p,k)
        predictions.append(pred_l)
    
    TP,FP,TN,FN = TP_FP_TN_FN(predictions,y_test)
    TP_Counter = Counter(TP)
    FP_Counter = Counter(FP)
    TN_Counter = Counter(TN)
    FN_Counter = Counter(FN)

    Ma_Acc,Ma_Pres,Ma_Rec,Ma_F1 = Macro_Labels(TP_Counter,FP_Counter,TN_Counter,FN_Counter,n)
    Mi_Acc,Mi_Pres,Mi_Rec,Mi_F1 = Micro_Labels(TP_Counter,FP_Counter,TN_Counter,FN_Counter,n)
    Accuracies,Precisions,Recalls,F1s = get_per_label(TP_Counter,FP_Counter,TN_Counter,FN_Counter,n)
    ex_precision,ex_recall,ex_accuracy,ex_F1 = precision_recall(predictions,y_test)
    
    return Accuracies,Precisions,Recalls,F1s,Mi_Acc,Mi_Pres,Mi_Rec,Mi_F1,ex_precision,ex_recall,ex_accuracy,ex_F1,Ma_Acc,Ma_Pres,Ma_Rec,Ma_F1

In [17]:
#def write_Accuracy(predictions,Y_Test,k,p,number,mlb):
#    Accuracies,Precisions,Recalls,F1s,Mi_Acc,Mi_Pres,Mi_Rec,Mi_F1,ex_precision,ex_recall,ex_accuracy,ex_F1,Ma_Acc,Ma_Pres,Ma_Rec,Ma_F1 = get_recall_at_K_Example(predictions,Y_Test,k,p,len(mlb.classes_))#

#    row = ["Classes","Accuracy","Precision","Recall","F1-Score"]
#    rows = []
#    rows.append(row)
#    for i in range(len(mlb.classes_)):
#        row  = [mlb.classes_[i],Accuracies[i],Precisions[i],Recalls[i],F1s[i]]
#        rows.append(row)    
#    
#    rows.append(["MACRO AVG",Ma_Acc,Ma_Pres,Ma_Rec,Ma_F1])
#    rows.append(["Micro AVG",Mi_Acc,Mi_Pres,Mi_Rec,Mi_F1])
#    rows.append(["EXAMPLES",ex_accuracy,ex_precision,ex_recall,ex_F1])
#    location = "GeometricReports/"+str(number)+".csv"
#    with open(location,"a",newline="") as f1: 
#        cw = csv.writer(f1)
#        cw.writerows(rows)
#    f1.close()
                

In [18]:
def csv_generator_OLD(X_Train,Y_Train,bs,mlb,mode):
    i = 0
    while True:
        x_features = []
        y_features = []
        x = 0 
        while x < bs:
            if i >= len(X_Train):
                i = 0
                if mode == "eval":
                    break
            if i < len(X_Train):
                x_features.append(X_Train[i])
                y_features.append(Y_Train[i])
                i+=1
            else:
                i+=1
                continue
            x +=1
            
        y_features = np.array(y_features)
        y_features = mlb.transform(y_features)
        yield(np.array(x_features),y_features)

In [27]:
def get_category_encoder(train_path_geo,test_path_geo,valid_path_geo):
    #Read the list 
    with open(train_path_geo, 'r') as f1:
        reader = csv.reader(f1)
        your_list = list(reader)
    f1.close()

    #Read the list 
    with open(test_path_geo, 'r') as f1:
        reader = csv.reader(f1)
        your_list_test = list(reader)
    f1.close()

    #Read the list 
    with open(valid_path_geo, 'r') as f1:
        reader = csv.reader(f1)
        your_list_valid = list(reader)
    f1.close()
    
    #Encoders
    min_max_scaler = preprocessing.MinMaxScaler()
    mlb = MultiLabelBinarizer()
    enc_category = OneHotEncoder()

    C_Train = []
    X_Train = []
    Y_Train = []
    for i in range(len(your_list)):
        y1 = ast.literal_eval(your_list[i][0])
        y1 = [n.strip() for n in y1]
        Y_Train.append(y1)
        
        g2 = your_list[i][1]
        g3 = your_list[i][2]
        C_Train.append([g2,g3])
        x1 = [float(your_list[i][3]),float(your_list[i][4]),float(your_list[i][5]),
              float(your_list[i][6]),float(your_list[i][7]),float(your_list[i][8]),
              float(your_list[i][10]),float(your_list[i][11]),float(your_list[i][12]),
              float(your_list[i][13]),float(your_list[i][14]),float(your_list[i][15])]
        X_Train.append(x1)
        
    del your_list[:]

    C_Test = []
    X_Test = []
    Y_Test = []
    for i in range(len(your_list_test)):
        y1 = ast.literal_eval(your_list_test[i][0])
        y1 = [n.strip() for n in y1]
        Y_Test.append(y1)
        
        g2 = your_list_test[i][1]
        g3 = your_list_test[i][2]
        C_Test.append([g2,g3])

    del your_list_test[:]

    C_Valid = []
    Y_Valid = []
    for i in range(len(your_list_valid)):
        y1 = ast.literal_eval(your_list_valid[i][0])
        y1 = [n.strip() for n in y1]
        Y_Valid.append(y1)

        g2 = your_list_valid[i][1]
        g3 = your_list_valid[i][2]
        C_Valid.append([g2,g3])

    del your_list_valid[:]
    #Fitting the data
    Out =  C_Train+C_Test+C_Valid
    enc_category.fit(Out)
    mlb.fit(Y_Train)
    mlb.fit(Y_Valid)
    mlb.fit(Y_Test)
    min_max_scaler.fit(X_Train)
    
    TX = len(X_Train)
    TV = len(Y_Valid)
    TT = len(Y_Test)
    del Out[:]
    
    return enc_category,min_max_scaler,mlb,TX,TV,TT,Y_Test

In [41]:
def csv_generator(inputPath, bs, mlb,enc_category,min_max_scaler):
    
    enc_compass = OneHotEncoder()
    enc_compass.fit([["North"],["East"],["South"],["West"]])
    # open the CSV file for reading
    f = open(inputPath, "r")
    while True:

        # initialize our batches of images and labels
        x_features = []
        y_features = []
        while len(x_features) < bs:

            X_Train = []
            E_Train = []
            C_Train = []
            
            line = f.readline()
            if line == "":
                f.seek(0)
                line = f.readline()

            empty_string = ""
            comma_counter = 1
            for x in line.rstrip():
                if x == "\"":
                    continue
                elif x == ",":
                    comma_counter += 1
                    empty_string = empty_string+x
                elif x == "]":
                    empty_string = empty_string+x
                    break
                else:
                    empty_string = empty_string+x
            
            line = line.strip().split(",")
            x_feature = [x1 for x1 in line[comma_counter:]]

            y1 = ast.literal_eval(empty_string)
            y1 = [n for n in y1]
            y_features.append(y1)

            g1 = enc_compass.transform([[x_feature[8]]]).toarray()
            g2 = x_feature[0]
            g3 = x_feature[1]
            C_Train.append([g2,g3])
            x1 = [float(x_feature[2]),float(x_feature[3]),float(x_feature[4]),
                  float(x_feature[5]),float(x_feature[6]),float(x_feature[7]),
                  float(x_feature[9]),float(x_feature[10]),float(x_feature[11]),
                  float(x_feature[12]),float(x_feature[13]),float(x_feature[14])]

            X_Train.append(x1)
            X_Train = min_max_scaler.transform(X_Train)
            C_Train = enc_category.transform(C_Train).toarray()
            E_Train.append(g1[0])

            X_Train = np.concatenate([X_Train, E_Train,C_Train], axis=1) 
            x_features.append(X_Train[0])

        y_features = np.array(y_features)
        y_features = mlb.transform(y_features)
        yield(np.array(x_features),y_features)

In [36]:
def csv_generator_test(inputPath, bs, mlb,enc_category,min_max_scaler):
    
    enc_compass = OneHotEncoder()
    enc_compass.fit([["North"],["East"],["South"],["West"]])
    # open the CSV file for reading
    f = open(inputPath, "r")
    i = 0
    while True:

        # initialize our batches of images and labels
        x_features = []
        while len(x_features) < 1:
            X_Train = []
            E_Train = []
            C_Train = []
            
            line = f.readline()
            #if line == "":
            #    f.seek(0)
            #    line = f.readline()

            empty_string = ""
            comma_counter = 1
            for x in line.rstrip():
                if x == "\"":
                    continue
                elif x == ",":
                    comma_counter += 1
                    empty_string = empty_string+x
                elif x == "]":
                    empty_string = empty_string+x
                    break
                else:
                    empty_string = empty_string+x

            line = line.strip().split(",")
            x_feature = [x1 for x1 in line[comma_counter:]]

            g1 = enc_compass.transform([[x_feature[8]]]).toarray()
            g2 = x_feature[0]
            g3 = x_feature[1]
            C_Train.append([g2,g3])
            x1 = [float(x_feature[2]),float(x_feature[3]),float(x_feature[4]),
                  float(x_feature[5]),float(x_feature[6]),float(x_feature[7]),
                  float(x_feature[9]),float(x_feature[10]),float(x_feature[11]),
                  float(x_feature[12]),float(x_feature[13]),float(x_feature[14])]

            X_Train.append(x1)
            X_Train = min_max_scaler.transform(X_Train)
            C_Train = enc_category.transform(C_Train).toarray()
            E_Train.append(g1[0])

            X_Train = np.concatenate([X_Train, E_Train,C_Train], axis=1) 
            x_features.append(X_Train[0])

        yield(np.array(x_features))

In [37]:
def run_10_geometric():
    for number in range(0,10):
        train_path_geo =  "Voc2kGeometric/Training_data_"+str(number)+".csv"
        test_path_geo =  "Voc2kGeometric/Testing_data_"+str(number)+".csv"
        valid_path_geo =  "Voc2kGeometric/Validation_data_"+str(number)+".csv"
        bs = 32
        enc_category,min_max_scaler,mlb,TX,TV,TT,Y_Test =  get_category_encoder(train_path_geo,test_path_geo,valid_path_geo)
        

        model = Sequential([
            Dense(2048, init="uniform",activation='relu', input_shape=(168,)),
            Dense(1024, kernel_initializer="uniform" ,activation='relu'),
            Dense(len(mlb.classes_), activation='sigmoid'),
        ])
        
        trainGen = csv_generator(train_path_geo, bs, mlb,enc_category,min_max_scaler)
        valGen   = csv_generator(valid_path_geo, bs, mlb,enc_category,min_max_scaler)
        #testGen  = csv_generator(test_path_geo, bs, mlb,enc_category,min_max_scaler)
        testGen  = csv_generator_test(test_path_geo, bs, mlb,enc_category,min_max_scaler)
        model.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['categorical_accuracy'])

        hist = model.fit_generator(trainGen, steps_per_epoch = (TX/32)-1,validation_data=valGen, validation_steps=(TV/32)-1,epochs=5)
        model_name = "GeometricModels/sv"+str(number)+"5_.h5"
        model.save(model_name)
        #evaluate_model(shape,mlb,BS,model,NUM_TEST_IMAGES,testLabels,TEST_CSV,number)
        #print("RUNNING PREDICTIONS")
        predictions = model.predict_generator(testGen, steps=TT/1,workers=1,use_multiprocessing=True, verbose=1)
        Y_Test = mlb.transform(Y_Test)
        #write_Accuracy(predictions,Y_Test,5,50,number,mlb)
        write_Accuracy(predictions,Y_Test,number,mlb)
        K.clear_session()

In [40]:
run_10_geometric()

/home/nuclearvodka/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2048, activation="relu", input_shape=(168,), kernel_initializer="uniform")`
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/5
['derriere', 'pres_de']
['derriere', 'pres_de']['devant', 'loin_de']

['devant', 'a_cote_de', 'pres_de']['a_cote_de', 'au_niveau_de', 'pres_de']

['a_cote_de', 'au_niveau_de', 'pres_de']
['sous']
['derriere', 'pres_de']
['derriere']
['sous', 'contre']
['au_niveau_de', 'pres_de']
['derriere']['sur', 'contre']

['au_niveau_de']
['devant']['a_cote_de', 'au_niveau_de', 'pres_de']

['en_face_de', 'pres_de']
['derriere']
['derriere', 'devant']['derriere', 'pres_de']

['sous']['derriere', 'au_niveau_de', 'pres_de']

['derriere', 'pres_de']
['devant', 'contre']
['au_dessus_de', 'a_cote_de', 'pres_de']['devant', 'pres_de']

['loin_de']['sur']

['derriere', 'contre']
['pres_de', 'devant']
['sur', 'contre']['a_cote_de', 'au_niveau_de', 'pres_de']

['sur', 'contre']
['pres_de', 'a_cote_de']['sur']

['sur']
['devant', 'derriere']
['a_cote_de', 'au_niveau_de', 'pres_de']
['pres_de', 'au_niveau_de']
['derriere', 'en_face_de']
['derriere', 'pres_de']
['devant', 'loin_de']['devant', 'pres_de'

 1/96 [..............................] - ETA: 1:07 - loss: 0.6942 - categorical_accuracy: 0.0312

SyntaxError: EOL while scanning string literal (<unknown>, line 1)

In [42]:
run_10_geometric()

/home/nuclearvodka/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2048, activation="relu", input_shape=(168,), kernel_initializer="uniform")`
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/5
 1/96 [..............................] - ETA: 1:02 - loss: 0.6912 - categorical_accuracy: 0.1250

SyntaxError: EOL while scanning string literal (<unknown>, line 1)